In [3]:
import sys
sys.path.append("/Users/alex/Downloads/stats/")

import pandas as pd
import os
from db import connection
from db_utils import DBUtils
import xlrd
from tqdm import tqdm
import datetime
import json
from glob import glob

In [4]:
values = ['UN WPP - Total population (both sexes combined) by single age, region, subregion and country, annually for 1950-2100 (thousands)',
        'UN WPP - Male population by single age, region, subregion and country, annually for 1950-2100 (thousands)',
         'UN WPP - Female population by single age, region, subregion and country, annually for 1950-2100 (thousands)',
         'UN WPP - Interpolated demographic indicators by region, subregion and country, annually for 1950-2099',
          'UN WPP - Interpolated total population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)',
          'UN WPP - Interpolated male population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)',
          'UN WPP - Interpolated female population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)',
          'UN WPP - Percentage of total population by broad age group, region, subregion and country, annually interpolated for 1950-2100',
          'UN WPP - Percentage of male total population by broad age group, region, subregion and country, annually interpolated for 1950-2100',
          'UN WPP - Percentage of female total population by broad age group, region, subregion and country, annually interpolated for 1950-2100',
          'UN WPP - Dependency ratios (total, child, old-age) for different age groups and for both sexes combined by region, subregion and country, annually interpolated for 1950-2100',
          'UN WPP - Male dependency ratios (total, child, old-age) for different age groups by region, subregion and country, annually interpolated for 1950-2100',
          'UN WPP - Female dependency ratios (total, child, old-age) for different age groups by region, subregion and country, annually interpolated for 1950-2100'
         ]

keys = ['Annual population by single age - Both Sexes. De facto population as of 1 July of the year indicated classified by single age (0, 1, 2, ..., 99, 100+). Data are presented in thousands.',
       'Annual population by single age - Male. De facto population as of 1 July of the year indicated classified by single age (0, 1, 2, ..., 99, 100+). Data are presented in thousands.',
       'Annual population by single age - Female. De facto population as of 1 July of the year indicated classified by single age (0, 1, 2, ..., 99, 100+). Data are presented in thousands.',
       'Annually interpolated demographic indicators.',
       'Annual total population (both sexes combined) by broad age groups. Data are presented in thousands.',
       'Annual male population by broad age groups. Data are presented in thousands.',
        'Annual female population by broad age groups. Data are presented in thousands.',
        'Percentage of annual total population (both sexes combined) by broad age group.',
        'Percentage of annual male total population by broad age groups.',
        'Percentage of annual female total population by broad age groups.',
        'Annual dependency ratios (total, child, old-age) for different age groups and for both sexes combined.',
        'Annual male dependency ratios (total, child, old-age) for different age groups.',
        'Annual female dependency ratios (total, child, old-age) for different age groups.'
       ]
datasets_dict = {}
for x,y in zip(keys, values):
    datasets_dict[x] = y

In [5]:
# with connection as c:
#     db = DBUtils(c)
    
#     # upsert datasets
#     dataset_name_ids = {}
#     for f in tqdm(os.listdir('sources/')):
#         if f == '.DS_Store':
#             continue
#         data = pd.read_excel('sources/'+f)
#         val = data[data.columns[0]][8]
#         index_to_remove = val.find(":")
#         res = "UN WPP - " + val[index_to_remove+2:]
#         dataset_id = db.upsert_dataset(name=res, namespace="unwpp", user_id=15)
#         dataset_name_ids[res] = dataset_id
        
#     # upsert sources
    
#     dataset_to_source_ids = {}
#     source_name = "United Nations – Population Division (2019 Revision)"
#     for addInfo, dataset_name in datasets_dict.items():
#         description = {}
#         description['dataPublishedBy'] = "United Nations, Department of Economic and Social Affairs, Population Division (2019). World Population Prospects: The 2019 Revision, DVD Edition."
#         description['dataPublisherSource'] = None
#         description['link'] = 'https://population.un.org/wpp2019/Download/Standard/Interpolated/'
#         description['retrievedDate'] = datetime.datetime.now().strftime("%d-%b-%Y")
#         description['additionalInfo'] = addInfo
        
#         source_id = db.upsert_source(name=source_name, description=json.dumps(description), dataset_id=dataset_name_ids[dataset_name])
#         dataset_to_source_ids[dataset_name_ids[dataset_name]] = source_id
    

In [6]:
# dataset_to_source_ids

# Preparing csvs

## Datasets

In [7]:
names,ids = [], []
i = 0
for f in tqdm(glob('data/*.xlsx')):
    
        data = pd.read_excel(f)
        val = data[data.columns[0]][8]
        index_to_remove = val.find(":")
        res = "UN WPP - " + val[index_to_remove+2:]
        names.append(res)
        ids.append(i)
        i+=1
datasets = pd.DataFrame()
datasets['id'] = ids
datasets['name'] = names
datasets.to_csv('datasets.csv', index=False)

100%|██████████| 13/13 [04:45<00:00, 20.77s/it]


## Sources

In [8]:
names, desc, d_ids = [], [], []

source_name = "United Nations – Population Division (2019 Revision)"
for addInfo, dataset_name in datasets_dict.items():
    description = {}
    description['dataPublishedBy'] = "United Nations, Department of Economic and Social Affairs, Population Division (2019). World Population Prospects: The 2019 Revision, DVD Edition."
    description['dataPublisherSource'] = None
    description['link'] = 'https://population.un.org/wpp2019/Download/Standard/Interpolated/'
    description['retrievedDate'] = datetime.datetime.now().strftime("%d-%b-%Y")
    description['additionalInfo'] = addInfo
    dataset_id = datasets[datasets['name'] == dataset_name]['id'].values[0]

    names.append(source_name)
    desc.append(description)
    d_ids.append(dataset_id)
    
res = pd.DataFrame()
res['name'] = names
res['description'] = desc
res['dataset_id'] = d_ids
res.to_csv("sources.csv", index=False)

## Variables

In [9]:
class DataVariables():
    
    def __init__(self):
        
        
        self.i = 0
        self.ids = []
        self.names = []
        self.units = []
        self.dataset_ids = []
        self.doc_to_unit = {
                        "UN WPP - Female population by single age, region, subregion and country, annually for 1950-2100 (thousands)": "Thousands",
                        "UN WPP - Male population by single age, region, subregion and country, annually for 1950-2100 (thousands)": "Thousands",
            "UN WPP - Total population (both sexes combined) by single age, region, subregion and country, annually for 1950-2100 (thousands)": "Thousands",
            "UN WPP - Female dependency ratios (total, child, old-age) for different age groups by region, subregion and country, annually interpolated for 1950-2100": "Female dependency ratios for different age groups",
            "UN WPP - Male dependency ratios (total, child, old-age) for different age groups by region, subregion and country, annually interpolated for 1950-2100": "Male dependency ratios for different age groups",
            "UN WPP - Dependency ratios (total, child, old-age) for different age groups and for both sexes combined by region, subregion and country, annually interpolated for 1950-2100": "Dependency ratios (both sexes combined) for different age groups",
            "UN WPP - Percentage of female total population by broad age group, region, subregion and country, annually interpolated for 1950-2100": "Percentage",
            "UN WPP - Percentage of male total population by broad age group, region, subregion and country, annually interpolated for 1950-2100": "Percentage",
            "UN WPP - Percentage of total population by broad age group, region, subregion and country, annually interpolated for 1950-2100": "Percentage",
            "UN WPP - Interpolated female population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)": "Thousands",
            "UN WPP - Interpolated male population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)": "Thousands",
            "UN WPP - Interpolated total population by broad age group, region, subregion and country, annually for 1950-2100 (thousands)": "Thousands"
            #UN WPP - Interpolated demographic indicators by region, subregion and country, annually for 1950-2099
                   

                    }
        
        self.datasets = pd.read_csv('datasets.csv')
        self.df = pd.DataFrame()
    
        
    def get_variables(self, path, skiprows=8, prefix=None):
        
        for sh in ["ESTIMATES", "MEDIUM VARIANT"]:
            data = pd.read_excel(path, skiprows=skiprows, sheet_name=sh)

            val = data[data.columns[0]][0]
            index_to_remove = val.find(":")
            res = "UN WPP - " + val[index_to_remove+2:]

            title = data[data.columns[0]][1]
            print(title)

            for item in data.loc[7, data.columns[8]: data.columns[-1]].values:
                if prefix:
                    self.names.append(title + ": " + prefix + " - " + item)
                else:
                    self.names.append(title + ": " + item)
                self.ids.append(self.i)
                self.i+=1
                self.units.append(self.doc_to_unit[res])
                self.dataset_ids.append(self.datasets[self.datasets['name'] == res]['id'].values[0])
    
    def get_custom_variable(self, path, skiprows=8, prefix=None):
        
        column_unit = {
            8: "Thousands",
            9: "Thousands",
            10: "Thousands",
            11: "Percentage",
            12: "Years",
            13: "Years",
            14: "Years",
            15: "Thousands",
            16: "Infant deaths per 1,000 live births",
            17: "Deaths under age 5 per 1,000 live births",
            18: "Thousands",
            19: "Births per 1,000 population",
            20: "Live births per woman",
            21: "Thousands",
            22: "Per 1,000 population",
            23: "Thousands",
            24: "Percentage"
            
        
        }
        
        for sh in ["ESTIMATES", "MEDIUM VARIANT"]:
            data = pd.read_excel(path, skiprows=skiprows, sheet_name=sh)

            val = data[data.columns[0]][0]
            index_to_remove = val.find(":")
            res = "UN WPP - " + val[index_to_remove+2:]

            title = data[data.columns[0]][1]
            print(title)
            
            col_index = 8
            for item in data.loc[7, data.columns[8]: data.columns[-1]].values:
                if prefix:
                    self.names.append(title + ": " + prefix + " - " + item)
                else:
                    self.names.append(title + ": " + item)
                self.ids.append(self.i)
                self.i+=1
                self.units.append(column_unit[col_index])
                self.dataset_ids.append(self.datasets[self.datasets['name'] == res]['id'].values[0])
                col_index += 1
           
    

            
            
    def get_df(self):
        
        self.df['id'] = self.ids
        self.df['name'] = self.names
        self.df['unit'] = self.units
        self.df['dataset_id'] = self.dataset_ids
        
        return self.df
        
        
        

In [33]:
name_to_prefix = {
    "data/WPP2019_INT_F03_3_POPULATION_BY_AGE_ANNUAL_FEMALE.xlsx": "Female population by age (thousands)",
    "data/WPP2019_INT_F03_2_POPULATION_BY_AGE_ANNUAL_MALE.xlsx": "Male population by age (thousands)",
    "data/WPP2019_INT_F03_1_POPULATION_BY_AGE_ANNUAL_BOTH_SEXES.xlsx": "Total population by age, both sexes combined (thousands)",
    "data/WPP2019_INT_F02C_3_ANNUAL_POPULATION_INDICATORS_DEPENDENCY_RATIOS_FEMALE.xlsx": "Female dependency ratios for different age groups",
    "data/WPP2019_INT_F02C_2_ANNUAL_POPULATION_INDICATORS_DEPENDENCY_RATIOS_MALE.xlsx": "Male dependency ratios for different age groups",
    "data/WPP2019_INT_F02C_1_ANNUAL_POPULATION_INDICATORS_DEPENDENCY_RATIOS_BOTH_SEXES.xlsx": "Dependency ratios (both sexes combined) for different age groups",
    "data/WPP2019_INT_F02B_3_ANNUAL_POPULATION_INDICATORS_PERCENTAGE_FEMALE.xlsx": "Percentage of female population by broad age group (per 100 female total population)",
    "data/WPP2019_INT_F02B_2_ANNUAL_POPULATION_INDICATORS_PERCENTAGE_MALE.xlsx": "Percentage of male population by broad age group (per 100 male total population)",
    "data/WPP2019_INT_F02B_1_ANNUAL_POPULATION_INDICATORS_PERCENTAGE_BOTH_SEXES.xlsx": "Percentage of total population by broad age group, both sexes combined (per 100 total population)",
    "data/WPP2019_INT_F02A_3_ANNUAL_POPULATION_INDICATORS_FEMALE.xlsx": "Female population by broad age group (thousands)",
    "data/WPP2019_INT_F02A_2_ANNUAL_POPULATION_INDICATORS_MALE.xlsx": "Male population by broad age group (thousands)",
    "data/WPP2019_INT_F02A_1_ANNUAL_POPULATION_INDICATORS_BOTH_SEXES.xlsx": "Total population by broad age group, both sexes combined (thousands)"
}

In [34]:
dv = DataVariables()
for f in tqdm(glob('data/*.xlsx')):
     
        if f == "data/WPP2019_INT_F01_ANNUAL_DEMOGRAPHIC_INDICATORS.xlsx":
            dv.get_custom_variable(f, prefix="Annually interpolated demographic indicators")
        else:
            dv.get_variables(f, prefix=name_to_prefix[f])    
    
        

res = dv.get_df()









  0%|          | 0/13 [00:00<?, ?it/s]

Estimates, 1950 - 2020









  8%|▊         | 1/13 [00:20<04:03, 20.33s/it]

Medium fertility variant, 2020 - 2100
Estimates, 1950 - 2020









 15%|█▌        | 2/13 [01:07<05:10, 28.26s/it]

...

Medium fertility variant, 2020 - 2100
Estimates, 1950 - 2020









 92%|█████████▏| 12/13 [08:52<00:41, 41.46s/it]

Medium fertility variant, 2020 - 2100
Estimates, 1950 - 2020









100%|██████████| 13/13 [09:36<00:00, 42.14s/it]

Medium fertility variant, 2020 - 2100


In [35]:
res.to_csv("variables.csv", index=False)

## Datapoints

In [28]:
def normalize_country(row):
        
        row['country'] = row['country'].str.replace(r'\s*[^A-Za-z\s]*$', '')
        return row


def get_variables(path, variables, skiprows=8, prefix=None, custom=False):
        
        for sh in ["ESTIMATES", "MEDIUM VARIANT"]:
            data = pd.read_excel(path, skiprows=skiprows, sheet_name=sh)

            val = data[data.columns[0]][0]
            index_to_remove = val.find(":")
            res = "UN WPP - " + val[index_to_remove+2:]

            title = data[data.columns[0]][1]
            
            index_col = 8
            
            for item in data.loc[7, data.columns[8]: data.columns[-1]].values:
                
                if prefix:
                    var_name = title + ": " + prefix + " - " + item
                else:
                    var_name = title + ": " + item
                
                var_id = variables[variables['name'] == var_name]['id'].values[0] 
                
                data2 = data.iloc[8:]
               
                data_res = pd.DataFrame()
                data_res['country'] = data2[data2.columns[2]]
                data_res['year'] = data2[data2.columns[7]]
                data_res['value'] = data2["Unnamed: %s" % str(index_col)]
                
                data_res['country'] = normalize_country(data_res)
                data_res = data_res[data_res['value'] != '...']
                data_res.to_csv('datapoints/datapoints_%s.csv' % str(var_id), index=False)
                
                index_col += 1
            

In [36]:
variables = pd.read_csv("variables.csv")

for f in tqdm(glob('data/*.xlsx')):
   
    if f == "data/WPP2019_INT_F01_ANNUAL_DEMOGRAPHIC_INDICATORS.xlsx":
        get_variables(f,variables, prefix="Annually interpolated demographic indicators")
    else:
        get_variables(f,variables, prefix=name_to_prefix[f])  
   








  0%|          | 0/13 [00:00<?, ?it/s]






  8%|▊         | 1/13 [00:22<04:25, 22.17s/it]






 15%|█▌        | 2/13 [01:15<05:47, 31.61s/it]






 23%|██▎       | 3/13 [02:47<08:15, 49.60s/it]






 31%|███       | 4/13 [04:18<09:17, 61.96s/it]






 38%|███▊      | 5/13 [04:37<06:34, 49.29s/it]






 46%|████▌     | 6/13 [05:31<05:53, 50.53s/it]






 54%|█████▍    | 7/13 [05:53<04:12, 42.03s/it]






 62%|██████▏   | 8/13 [07:24<04:43, 56.73s/it]






 69%|██████▉   | 9/13 [08:17<03:42, 55.62s/it]






 77%|███████▋  | 10/13 [08:39<02:16, 45.55s/it]






 85%|████████▍ | 11/13 [09:32<01:35, 47.82s/it]






 92%|█████████▏| 12/13 [10:25<00:49, 49.30s/it]






100%|██████████| 13/13 [11:17<00:00, 50.25s/it]

## Get country names

In [17]:
# countries = set()

# for x in tqdm(glob('datapoints/*.csv')):
    
   
#     data = pd.read_csv(x)
#     for j in data['country'].values:
#         countries.add(j)
# res = pd.DataFrame()
# res['name'] = list(countries)
# res.to_csv("distinct_countries_standardized.csv", index=False)


100%|██████████| 356/356 [00:06<00:00, 51.32it/s]

True

## Insert db

In [37]:
with connection as c:
    db = DBUtils(c)
    
    entities = pd.read_csv("distinct_countries_standardized.csv")
    datasets = pd.read_csv("datasets.csv")
    sources = pd.read_csv("sources.csv")
    variables = pd.read_csv('variables.csv')
    
    new_entities = entities[entities['db_entity_id'].isnull()]
    for _, entity in new_entities.iterrows():
        entity_id = entity.name
        entity_name = entity['name']
        db_entity_id = db.get_or_create_entity(entity_name)
        entities.loc[entity_id, 'db_entity_id'] = db_entity_id
    
    # upsert datasets
    dataset_name_ids = {}
    for i, row in tqdm(datasets.iterrows()):
        dataset_id = db.upsert_dataset(name=row['name'], namespace="unwpp", user_id=15)
        dataset_name_ids[row['name']] = dataset_id
        
        
    # upsert sources
    
    dataset_to_source_ids = {}
    for i, row in tqdm(sources.iterrows()):

        dataset_name = datasets[datasets['id'] == row['dataset_id']]['name'].values[0]
        source_id = db.upsert_source(name=row['name'], description=json.dumps(row['description']), dataset_id=dataset_name_ids[dataset_name])

        dataset_to_source_ids[dataset_name] = source_id

        
    # upsert variables
    names_to_ids = {}
    for i, row in tqdm(variables.iterrows()):
        
        dataset_name = datasets[datasets['id'] == row['dataset_id']]['name'].values[0]
        dataset_id = dataset_name_ids[dataset_name]
        source_id = dataset_to_source_ids[dataset_name]

        
        variable_id = db.upsert_variable(
                                        name=row['name'], 
                                        code=None, 
                                        unit=row['unit'], 
                                        short_unit=None, 
                                        source_id=source_id, 
                                        dataset_id=dataset_id, 
                                        description=None, 
                                        timespan='', 
                                        coverage='', 
                                        display={}
                                        )
        names_to_ids[row['name']] = variable_id
        
    #Inserting datapoints


    datapoints_files = glob("datapoints/*.csv")
    for x in tqdm(datapoints_files): 
        # to get variable is
        v_id = int(x.split("_")[1].split(".")[0])
       
        # to get variable name
        variable_name = variables[variables['id']==v_id]['name'].values[0]
       
        # to get variable id from db
        variable_id = names_to_ids[variable_name]
        data = pd.read_csv(x)

        for i, row in data.iterrows():
            entity_id = entities[entities['name'] == row['country']]['db_entity_id'].values[0]

            year = row['year']

            db.upsert_one("""
                INSERT INTO data_values
                    (value, year, entityId, variableId)
                VALUES
                    (%s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                    value = VALUES(value),
                    year = VALUES(year),
                    entityId = VALUES(entityId),
                    variableId = VALUES(variableId)
            """, [val, int(year), str(int(entity_id)), str(variable_id)])
    

InterfaceError: (0, '')